In [3]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

In [5]:
def data():
    # Split train & validation set
    FFF_Train_Decile10 = pd.read_pickle("FFF_Train_Decile10.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile10.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile10, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 10')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 17s 6ms/step - loss: 0.2275 - acc: 0.9106 - val_loss: 0.1902 - val_acc: 0.9319
Epoch 35/75
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2277 - acc: 0.9119 - val_loss: 0.1900 - val_acc: 0.9319
Epoch 36/75
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2155 - acc: 0.9123 - val_loss: 0.1901 - val_acc: 0.9319
Epoch 37/75
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2156 - acc: 0.9116 - val_loss: 0.1900 - val_acc: 0.9332
Epoch 38/75
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2260 - acc: 0.9136 - val_loss: 0.1896 - val_acc: 0.9346
Epoch 39/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2179 - acc: 0.9159 - val_loss: 0.1895 - val_acc: 0.9346
Epoch 40/75
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2239 - acc: 0.9106 - val_loss: 0.1895 - val_acc: 0.9332
Epoch 41/75
3055/3055 [==============================] - 16s

3055/3055 [==============================] - 16s 5ms/step - loss: 0.2032 - acc: 0.9205 - val_loss: 0.1921 - val_acc: 0.9280

Epoch 00018: ReduceLROnPlateau reducing learning rate to 4.9999996554106475e-05.
Epoch 19/100
3055/3055 [==============================] - 17s 6ms/step - loss: 0.2075 - acc: 0.9178 - val_loss: 0.1924 - val_acc: 0.9280
Epoch 20/100
3055/3055 [==============================] - 17s 6ms/step - loss: 0.2126 - acc: 0.9195 - val_loss: 0.1922 - val_acc: 0.9267
Epoch 21/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2076 - acc: 0.9116 - val_loss: 0.1926 - val_acc: 0.9280
Epoch 22/100
3055/3055 [==============================] - 18s 6ms/step - loss: 0.2028 - acc: 0.9188 - val_loss: 0.1929 - val_acc: 0.9293
Epoch 23/100
3055/3055 [==============================] - 17s 6ms/step - loss: 0.2125 - acc: 0.9178 - val_loss: 0.1930 - val_acc: 0.9280

Epoch 00023: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 24/100
3055/3055 [====================

Epoch 77/100
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2039 - acc: 0.9139 - val_loss: 0.1932 - val_acc: 0.9280
Epoch 78/100
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2057 - acc: 0.9185 - val_loss: 0.1931 - val_acc: 0.9280
Epoch 79/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2043 - acc: 0.9172 - val_loss: 0.1931 - val_acc: 0.9267
Epoch 80/100
3055/3055 [==============================] - 17s 6ms/step - loss: 0.2053 - acc: 0.9191 - val_loss: 0.1931 - val_acc: 0.9280
Epoch 81/100
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2085 - acc: 0.9152 - val_loss: 0.1931 - val_acc: 0.9280
Epoch 82/100
3055/3055 [==============================] - 18s 6ms/step - loss: 0.2068 - acc: 0.9231 - val_loss: 0.1932 - val_acc: 0.9293
Epoch 83/100
3055/3055 [==============================] - 18s 6ms/step - loss: 0.2039 - acc: 0.9188 - val_loss: 0.1931 - val_acc: 0.9280
Epoch 84/100
3055/3055 [=================

3055/3055 [==============================] - 15s 5ms/step - loss: 0.2117 - acc: 0.9142 - val_loss: 0.1780 - val_acc: 0.9332
Epoch 36/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2107 - acc: 0.9116 - val_loss: 0.1780 - val_acc: 0.9332
Epoch 37/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2111 - acc: 0.9133 - val_loss: 0.1781 - val_acc: 0.9332
Epoch 38/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2080 - acc: 0.9165 - val_loss: 0.1779 - val_acc: 0.9346
Epoch 39/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2140 - acc: 0.9126 - val_loss: 0.1779 - val_acc: 0.9346
Epoch 40/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2117 - acc: 0.9178 - val_loss: 0.1779 - val_acc: 0.9346
Epoch 41/100
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2140 - acc: 0.9123 - val_loss: 0.1780 - val_acc: 0.9346
Epoch 42/100
3055/3055 [==============================

...

3055/3055 [==============================] - 17s 5ms/step - loss: 0.4696 - acc: 0.8331 - val_loss: 0.4065 - val_acc: 0.8809
Epoch 7/50
3055/3055 [==============================] - 15s 5ms/step - loss: 0.4193 - acc: 0.8645 - val_loss: 0.3736 - val_acc: 0.9031
Epoch 8/50
3055/3055 [==============================] - 16s 5ms/step - loss: 0.4005 - acc: 0.8831 - val_loss: 0.3524 - val_acc: 0.8966
Epoch 9/50
3055/3055 [==============================] - 16s 5ms/step - loss: 0.3850 - acc: 0.8867 - val_loss: 0.3412 - val_acc: 0.8992
Epoch 10/50
3055/3055 [==============================] - 16s 5ms/step - loss: 0.3820 - acc: 0.8789 - val_loss: 0.3315 - val_acc: 0.8992
Epoch 11/50
3055/3055 [==============================] - 16s 5ms/step - loss: 0.3759 - acc: 0.8838 - val_loss: 0.3368 - val_acc: 0.8979
Epoch 12/50
3055/3055 [==============================] - 16s 5ms/step - loss: 0.3667 - acc: 0.8907 - val_loss: 0.3177 - val_acc: 0.8979
Epoch 13/50
3055/3055 [==============================] - 17s 5m

Epoch 16/75
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2651 - acc: 0.8998 - val_loss: 0.2086 - val_acc: 0.9280
Epoch 17/75
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2438 - acc: 0.9103 - val_loss: 0.2052 - val_acc: 0.9267
Epoch 18/75
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2451 - acc: 0.9126 - val_loss: 0.2030 - val_acc: 0.9267
Epoch 19/75
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2558 - acc: 0.9070 - val_loss: 0.2030 - val_acc: 0.9241
Epoch 20/75
3055/3055 [==============================] - 17s 6ms/step - loss: 0.2427 - acc: 0.9100 - val_loss: 0.2001 - val_acc: 0.9293
Epoch 21/75
3055/3055 [==============================] - 17s 6ms/step - loss: 0.2412 - acc: 0.9116 - val_loss: 0.1994 - val_acc: 0.9280
Epoch 22/75
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2407 - acc: 0.9067 - val_loss: 0.1999 - val_acc: 0.9280
Epoch 23/75
3055/3055 [=========================

3055/3055 [==============================] - 17s 6ms/step - loss: 0.2322 - acc: 0.9061 - val_loss: 0.1934 - val_acc: 0.9293
Test accuracy: 0.929319371728
Train on 3055 samples, validate on 764 samples
Epoch 1/100
3055/3055 [==============================] - 23s 8ms/step - loss: 0.4308 - acc: 0.8288 - val_loss: 0.2820 - val_acc: 0.9005
Epoch 2/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.3405 - acc: 0.8674 - val_loss: 0.2721 - val_acc: 0.9045
Epoch 3/100
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2979 - acc: 0.8933 - val_loss: 0.2664 - val_acc: 0.9058
Epoch 4/100
3055/3055 [==============================] - 17s 5ms/step - loss: 0.2847 - acc: 0.9002 - val_loss: 0.2352 - val_acc: 0.9097
Epoch 5/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2565 - acc: 0.8989 - val_loss: 0.2117 - val_acc: 0.9175
Epoch 6/100
3055/3055 [==============================] - 16s 5ms/step - loss: 0.2456 - acc: 0.9057 - val_loss: 0.2065 -

3055/3055 [==============================] - 11s 4ms/step - loss: 0.1942 - acc: 0.9227 - val_loss: 0.1910 - val_acc: 0.9306
Epoch 58/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2025 - acc: 0.9133 - val_loss: 0.1908 - val_acc: 0.9293
Epoch 59/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2021 - acc: 0.9155 - val_loss: 0.1908 - val_acc: 0.9293
Epoch 60/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2045 - acc: 0.9169 - val_loss: 0.1908 - val_acc: 0.9293
Epoch 61/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2074 - acc: 0.9165 - val_loss: 0.1907 - val_acc: 0.9293
Epoch 62/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2035 - acc: 0.9188 - val_loss: 0.1908 - val_acc: 0.9306
Epoch 63/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2001 - acc: 0.9172 - val_loss: 0.1908 - val_acc: 0.9306
Epoch 64/100
3055/3055 [==============================

3055/3055 [==============================] - 11s 4ms/step - loss: 0.2194 - acc: 0.9155 - val_loss: 0.1840 - val_acc: 0.9306
Epoch 17/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2138 - acc: 0.9162 - val_loss: 0.1820 - val_acc: 0.9319
Epoch 18/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2196 - acc: 0.9126 - val_loss: 0.1823 - val_acc: 0.9372
Epoch 19/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2227 - acc: 0.9136 - val_loss: 0.1880 - val_acc: 0.9332
Epoch 20/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2223 - acc: 0.9116 - val_loss: 0.1823 - val_acc: 0.9372
Epoch 21/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2186 - acc: 0.9162 - val_loss: 0.1832 - val_acc: 0.9372
Epoch 22/50
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2135 - acc: 0.9172 - val_loss: 0.1822 - val_acc: 0.9359

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.

In [ ]:
0.93484